In [ ]:
!pip install mido
import mido
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
import pandas as pd

def list_files(startpath): #Prototype function for listing the directory
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
            
def load_midi(path): #Load midi file
    mid = mido.MidiFile(path)
    return mid

def bpm_to_tempo(bpm): #Converts tempo in beats/minute to microseconds/beat
    tempo = (60000000/bpm)
    return tempo

def tempo_to_bpm(tempo): #Converts tempo in microseconds/beat to beats/minute
    bpm = 60000000/tempo
    return bpm

def ticks_to_seconds(ticks,tempo,tpb):
    return tempo/1000000*ticks/tpb
    
print()

def quantize_track(track ,ticks_per): #ticks_per denotes quantization base. 12 for 32nds, 24 for 16ths and 48 for 8ths
    for msg in track:
        msg.time = round(msg.time/ticks_per)*ticks_per
    return track


def print_verbose(mid,length):
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track[0:length]:
            print(msg)
            
def print_track_verbose(midi_track):
    for msg in midi_track:
        print(msg)
        
def generate_pitch_classes(): #Returns a list where the first index (0-11) corresponds to each musical note starting from A0
    notes = [[],[],[],[],[],[],[],[],[],[],[],[]]
    A0 = 21
    for i in range(8):
        for j in range(12):
            note = A0 +i*12 + j
            if note <= 108:
                notes[j].append(note)
    return notes

def note_to_pitchclass(midi_note,pitch_classes, with_paus = False): #Returns a vector with a 1 for the entered note
    pitch_class = [0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(12):
        if midi_note in pitch_classes[i]:
            pitch_class[i] = 1
    return pitch_class

pitch_classes = generate_pitch_classes()


In [ ]:
classes = list(range(21,128))
classes.insert(0,0)
'''
def read_txt_files(data_folder):
    folds = []
    for file in sorted(os.listdir(data_folder)):
        if not file.split('.')[1] == 'txt': #Read only txt files
            continue
        with open(data_folder + '/' + file,'r') as f: #Read groups one at a time
            groups = []
            for line in f:
                group = []
                for event in line.split(','):
                    event = event.split('_')
                    note = event[0]
                    vel = event[1]
                    time = event[2]
                    pitch_class = note_to_pitchclass(int(note),pitch_classes)
                    attr = [int(note), int(vel), int(time)]
                    attr.extend(pitch_class)
                    group.append(attr)
                groups.append(group)
        folds.append(groups)
    return folds
'''
def note_class(note_value,vel,classes):
    class_vector = np.zeros((1,2*108))
    if vel > 0:
      class_vector[0,classes.index(note_value)*2] = 1
    else:
      class_vector[0,classes.index(note_value)*2+1] = 1
    return class_vector.tolist()[0]

def class_vector_to_note(vector,classes):
    idx = np.argwhere(vector == 1)[0][0]
    if idx % 2 == 0:
      vel = 100
      note = idx/2
    else:
      vel = 0
      note = round(idx/2)
    return note, vel


def read_train_txt(train_txt):
  path = train_txt
  with open(path,'r') as f: #Read groups one at a time
    note_groups = []
    time_groups = []
    for i,line in enumerate(f):
      note_group = []
      time_group = []
      for event in line.split(','):
        event = event.split('_')
        note = int(event[0])
        vel = int(event[1])
        time = int(event[2])
        n_class = note_class(note,vel,classes)                                             
        note_groups.append(np.array(n_class))
        time_groups.append(time)
          
    return note_groups, time_groups
train_note, train_time = read_train_txt('train.txt')







In [ ]:
import random
def random_training_set(data, batch_size,chunk_len, input_size, is_cuda=True, data_type = 0):
    file_len = len(data)
    inp = torch.Tensor(batch_size, chunk_len, input_size)
    target = torch.Tensor(batch_size, 1, input_size)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len
        chunk = data[start_index:end_index]
        if data_type:
          chunk = [[i] for i in chunk]
        inp[bi] = torch.Tensor(chunk)
        if data_type:
          target[bi] = torch.Tensor([data[end_index]])
        else:
          target[bi] = torch.Tensor(data[end_index])
    inp = torch.Tensor(inp)
    target = torch.Tensor(target)
    if is_cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target
#test_inp, test_target = random_training_set(train_note, 10,100,216)



In [ ]:
def array_to_midi(data,midi_name):
    
    mid = mido.MidiFile()
    mid.ticks_per_beat = 384
    track0 = mido.MidiTrack()
    track1 = mido.MidiTrack()

    track0.append(mido.MetaMessage('set_tempo', tempo = 500000, time = 0)) # Meta information like tempo and time signature assigned to track 0
    track0.append(mido.MetaMessage('time_signature', numerator = 4, denominator = 4, clocks_per_click = 24, notated_32nd_notes_per_beat = 8, time = 0))
    track0.append(mido.MetaMessage('end_of_track', time = 1))
    mid.tracks.append(track0)
    track1.append(mido.Message('program_change', channel = 0, program = 0, time = 0))
    
    for event in data: #Write notes to track 1
      note = event[0]
      vel = event[1]
      time = event[2]
      track1.append(mido.Message('note_on', channel = 0, note = note, velocity = vel, time = time))
    track1.append(mido.MetaMessage('end_of_track', time = 1))
    
    mid.tracks.append(track1)

    mid.save(midi_name)


    
    

In [ ]:
test_tensor = torch.Tensor(np.arange(0,10)).view(1,10,1)
print(torch.roll(test_tensor,-1,1).shape)

torch.Size([1, 10, 1])


In [ ]:
class Note_RNN(nn.Module):
    def __init__(self, input_size, rnn_size, hidden_size, output_size, seq_len, net_type, n_layers=1):
        super(Note_RNN, self).__init__()
        self.input_size = input_size
        self.net_type = net_type
        self.dropout = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.3)
        self.rnn_size = rnn_size
        self.batch_size = None
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.rnn = nn.GRU(input_size, rnn_size, n_layers, batch_first=True)
        self.fc1 = nn.Linear(rnn_size, hidden_size)
        self.rnn2 = nn.GRU(rnn_size, rnn_size, n_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size*seq_len, output_size)

    def forward(self, inp, rnn_hidden, rnn_hidden2):
        self.batch_size = inp.shape[0]
        out_dropout = self.dropout(inp)
        out_rnn1, hidden = self.rnn(out_dropout, rnn_hidden)
        out_dropout2 = self.dropout2(out_rnn1)
        out_rnn2, hidden2 = self.rnn2(out_dropout2, rnn_hidden2)
        out_fc1 = self.fc1(out_rnn2)
        out_dropout3 = self.dropout3(out_fc1)
        out_fc2 = self.fc2(out_dropout3.view(self.batch_size,seq_len*hidden_size))
        if self.net_type == 'note':
          output = F.softmax(out_fc2.view(self.batch_size,1,self.input_size),dim=2)
        elif self.net_type == 'time':
          output = F.relu(out_fc2.view(self.batch_size,1,self.input_size))
        return output, hidden, hidden2

    def init_hidden(self, batch_size):
        return torch.autograd.Variable(torch.zeros(self.n_layers, batch_size, self.rnn_size))
    




In [ ]:
 import datetime
def save(model,filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(model, save_filename)
    print('Saved as %s' % save_filename)
    
def train(epochs, batches_per_epoch, note_model_save_name, time_model_save_name, is_cuda=True):
  with open('Train_log_{}.txt'.format(datetime.datetime.now()), 'w') as f:
    print('Training for {} epochs'.format(epochs))
    note_net.train()
    hidden_note_rnn = note_net.init_hidden(batch_size)
    hidden_note_rnn2 = note_net.init_hidden(batch_size)
    hidden_time_rnn = time_net.init_hidden(batch_size)
    hidden_time_rnn2 = time_net.init_hidden(batch_size)
    if is_cuda:
        hidden_note_rnn = hidden_note_rnn.cuda()
        hidden_note_rnn2 = hidden_note_rnn2.cuda()
        hidden_time_rnn = hidden_time_rnn.cuda()
        hidden_time_rnn2 = hidden_time_rnn2.cuda()

    for epoch in range(epochs):
        epoch_loss_note = 0
        period_loss_note = 0
        epoch_loss_time = 0
        period_loss_time = 0

        for i in range(batches_per_epoch):
            inp, target = random_training_set(train_note, batch_size,seq_len,216)
            if is_cuda:
                inp = inp.cuda()
                target = target.cuda()
            note_net.zero_grad()
            note_optimizer.zero_grad()
            output_note, hidden_note_rnn, hidden_note_rnn2 = note_net(inp, hidden_note_rnn, hidden_note_rnn2)
            loss_note = criterion_note(output_note.squeeze(), torch.argmax(target.squeeze(),dim=1))
            loss_note.backward()
            note_optimizer.step()
            hidden_note_rnn = torch.autograd.Variable(hidden_note_rnn.data, requires_grad=True)
            hidden_note_rnn2 = torch.autograd.Variable(hidden_note_rnn2.data, requires_grad=True)
            epoch_loss_note = epoch_loss_note + loss_note.item()
            period_loss_note = period_loss_note + loss_note.item()
            if i % print_every == 0 and not i == 0:
                print('Epoch {}, Batch {}/{} Note Loss: {}'.format(epoch+1,i,batches_per_epoch,period_loss_note/(batch_size*print_every)))
                f.write('Epoch {}, Batch {}/{} Note Loss: {}\n'.format(epoch+1,i,batches_per_epoch,period_loss_note/(batch_size*print_every)))
                period_loss_note = 0
        print('---------------------------------------------------------------------------------------')
        f.write('---------------------------------------------------------------------------------------\n')
        for i in range(batches_per_epoch):
            inp, target = random_training_set(train_time, batch_size,seq_len,1,data_type=1)
            if is_cuda:
                inp = inp.cuda()
                target = target.cuda()
            time_net.zero_grad()
            time_optimizer.zero_grad()
            output_time, hidden_time_rnn, hidden_time_rnn2 = time_net(inp, hidden_time_rnn, hidden_time_rnn2)
            loss_time = criterion_time(output_time, target)
            loss_time.backward()
            time_optimizer.step()
            hidden_time_rnn = torch.autograd.Variable(hidden_time_rnn.data, requires_grad=True)
            hidden_time_rnn2 = torch.autograd.Variable(hidden_time_rnn2.data, requires_grad=True)
            epoch_loss_time = epoch_loss_time + loss_time.item()
            period_loss_time = period_loss_time + loss_time.item()
            if i % print_every == 0 and not i == 0:
                print('Epoch {}, Batch {}/{} Time Loss: {}'.format(epoch+1,i,batches_per_epoch,period_loss_time/(batch_size*print_every)))
                f.write('Epoch {}, Batch {}/{} Time Loss: {}\n'.format(epoch+1,i,batches_per_epoch,period_loss_time/(batch_size*print_every)))
                period_loss_time = 0
        #lr_scheduler_note.step()
        #lr_scheduler_time.step()
        print('Epoch {} total Note Loss: {}, total Time Loss: {}'.format(epoch+1, epoch_loss_note/(batch_size*batches_per_epoch), epoch_loss_time/(batch_size*batches_per_epoch)))
        f.write('Epoch {} total Note Loss: {}, total Time Loss: {}\n'.format(epoch+1, epoch_loss_note/(batch_size*batches_per_epoch), epoch_loss_time/(batch_size*batches_per_epoch)))
    save(note_net,note_model_save_name)
    save(time_net,time_model_save_name)
    f.write('Saving models....')
    f.write('Done!')
    f.close()

     

In [ ]:
import time
batch_size = 64
epochs = 1
hidden_size_rnn = 512
seq_len = 100
hidden_size = 256
learning_rate_note = 1e-5
learning_rate_time = 1e-5
batches_per_epoch = 1000
n_layers = 1
print_every = 100
is_cuda = True

note_net = Note_RNN(
    216,
    hidden_size_rnn,
    hidden_size,
    216,
    seq_len,
    'note',
    n_layers= n_layers,
)
time_net = Note_RNN(
    1,
    hidden_size_rnn,
    hidden_size,
    1,
    seq_len,
    'time',
    n_layers= n_layers,
)
note_optimizer = torch.optim.Adam(note_net.parameters(), lr=learning_rate_note)
criterion_note = nn.CrossEntropyLoss()
time_optimizer = torch.optim.Adam(time_net.parameters(), lr=learning_rate_time)
criterion_time = nn.MSELoss()
#lr_scheduler_note = torch.optim.lr_scheduler.StepLR(note_optimizer,
#                                               step_size=3,
#                                                gamma=0.1)
#lr_scheduler_time = torch.optim.lr_scheduler.StepLR(time_optimizer,
#                                               step_size=3,
#                                               gamma=0.1)
if is_cuda:
    note_net.cuda()
    time_net.cuda()

start = time.time()
train(epochs,batches_per_epoch,'Note_RNN_64_512_256_1e-5_1','Time_RNN_64_512_256_1e-5_1')
print('Total Training Time: {}'.format(time.time()-start))

Training for 1 epochs
Epoch 1, Batch 100/1000 Note Loss: 0.08479624412953854
Epoch 1, Batch 200/1000 Note Loss: 0.08385824531316757
Epoch 1, Batch 300/1000 Note Loss: 0.08384097591042519
Epoch 1, Batch 400/1000 Note Loss: 0.08384109929203987
Epoch 1, Batch 500/1000 Note Loss: 0.08382478170096874
Epoch 1, Batch 600/1000 Note Loss: 0.0838482304662466
Epoch 1, Batch 700/1000 Note Loss: 0.08384375363588333
Epoch 1, Batch 800/1000 Note Loss: 0.08383948557078838
Epoch 1, Batch 900/1000 Note Loss: 0.08382445611059666
---------------------------------------------------------------------------------------
Epoch 1, Batch 100/1000 Time Loss: 125.99932485580445
Epoch 1, Batch 200/1000 Time Loss: 94.69938794136047
Epoch 1, Batch 300/1000 Time Loss: 95.12005114555359
Epoch 1, Batch 400/1000 Time Loss: 242.1412124824524
Epoch 1, Batch 500/1000 Time Loss: 94.33013767242431
Epoch 1, Batch 600/1000 Time Loss: 88.79947328567505
Epoch 1, Batch 700/1000 Time Loss: 109.28015027999878
Epoch 1, Batch 800/1000

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Note_RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
def out_tensor_to_inp(in_tensor):
  out_tensor = torch.zeros((in_tensor.shape))
  max_idxs = torch.argmax(in_tensor,dim=2).tolist()
  for i,batch in enumerate(max_idxs):
    for j,idx in enumerate(batch):
      out_tensor[i,j,idx] = 1
  return out_tensor

def generate(note_net, time_net,num_notes, is_cuda=True,if_load=False):
  if if_load:
    note_net = torch.load(note_net)
    time_net = torch.load(time_net)
  out_notes = []
  out_times = []
  print('Generating....')
  hidden_note_rnn = note_net.init_hidden(1)
  hidden_note_rnn2 = note_net.init_hidden(1)
  hidden_time_rnn = note_net.init_hidden(1)
  hidden_time_rnn2 = time_net.init_hidden(1)
  if is_cuda:
    hidden_note_rnn = hidden_note_rnn.cuda()
    hidden_note_rnn2 = hidden_note_rnn2.cuda()
    hidden_time_rnn = hidden_time_rnn.cuda()
    hidden_time_rnn2 = hidden_time_rnn2.cuda()

  
  note_inp, _ = random_training_set(train_note, 1,100,216)



  time_inp, _ = random_training_set(train_time, 1,100,1, data_type = 1)

  #Initialize hidden layer
  if is_cuda:
    note_inp = note_inp.cuda()
    time_inp = time_inp.cuda()

  for i in range(num_notes):
    output_note, hidden_note_rnn, hidden_note_rnn2 = note_net(note_inp, hidden_note_rnn, hidden_note_rnn2)
    output_time, hidden_time_rnn, hidden_time_rnn2 = time_net(time_inp, hidden_time_rnn, hidden_time_rnn2)
    new_note = out_tensor_to_inp(output_note)
    new_time = round(float(output_time[0,0,0]))


    note_inp = torch.roll(note_inp,-1,1)[0,-1] = new_note
    np_time_inp = np.roll(time_inp.cpu().numpy(),-1,axis=1)
    print(time_inp)
'''
    time_inp = torch.roll(time_inp,-1,1)
    out_notes.append(new_note)
    out_times.append(new_time)

  out_data = []
  for i in range(len(out_notes)):
    note = out_notes[i].squeeze()
    time = out_times[i].cpu().squeeze().detach()
    note, vel = class_vector_to_note(note[j].numpy(),classes)
    event = [int(note), int(vel), int(time)]
    out_data.append(event)
  print('Creating Midi file')
  array_to_midi(out_data,'New_model_test.mid') 
  print(out_data[0])
'''
generate(note_net,time_net,100)

